In [ ]:
import requests
import json
import pandas as pd
import time
from bs4 import BeautifulSoup
from tqdm import tqdm


#불러올 페이지 수 (총 7222페이지)
i = 7222
df = pd.DataFrame()
df2 = pd.DataFrame()

for i in tqdm(range(1,(i+1))):
    requestURL = 'https://nedrug.mfds.go.kr/searchDrug?sort=&sortOrder=&searchYn=&ExcelRowdata=&page='+str(i)+'&searchDivision=detail&itemName=&entpName=&ingrName1=&ingrName2=&ingrName3=&itemSeq=&stdrCodeName=&atcCodeName=&indutyClassCode=&sClassNo=&narcoticKindCode=&cancelCode=&etcOtcCode=&makeMaterialGb=&searchConEe=AND&eeDocData=&searchConUd=AND&udDocData=&searchConNb=AND&nbDocData=&startPermitDate=&endPermitDate='
    while True:
        try:
            res = requests.get(requestURL)
            break;
        except:
            print("통신 에러")
            print("다시 request")
            continue;
    html_doc = res.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    #print(soup.prettify())
    
    #각 제품의 request보낼 url모으기
    itemSeqList=[]
    links = soup.find_all("a")
    for a in links:
        href = a.attrs['href']
        if "itemSeq" in href:
            #print(href)
            itemSeqList.append(href)
        itemNum = 15
        if i == 7222:
            itemNum = 14
        if len(itemSeqList) == itemNum:
            break
    #print(itemSeqList)
    
    #한 페이지당 15개 data가 있음
    #data 가져오기
    for j in range(0,len(itemSeqList)):
        cindex = []
        rindex = [str(j+1)]
        
        detailURL='https://nedrug.mfds.go.kr'+str(itemSeqList[j])
        while True:
            try:
                res2 = requests.get(detailURL)
                break;
            except:
                print("통신 에러")
                time.sleep(9)
                print("다시 request")
        html_doc2 = res2.text
        soup2 =BeautifulSoup(html_doc2, 'html.parser')
        #print(soup2.prettify())
            
        #col index 가져오기
        FILTER = ['년도','생산실적','순번','변경일자','변경항목'] #여기에 해당되는건 NO
        for title in soup2.find_all('th'):
            if(title.get_text() in (FILTER+cindex) ):
                break;
            cindex.append(title.get_text())
        datalist = []
        flag = 0
        for DATA in soup2.find_all('td'):
            #print(DATA.get_text())
            flag += 1
            datalist.append(DATA.get_text())
            if(flag == len(cindex)):
                break
                
        #유효성분 가져오기
        cindex.append("유효성분")
        ingredient = soup2.find_all('h3', class_='cont_title3 mt27 pb10')
        if ingredient:
            for ing in ingredient:
                datalist.append(ing.get_text().replace('유효성분 : ',''))
        else :
            datalist.append("")
            
        #유효성분 분량 가져오기
        #나중에 여기서 a[0]으로 세세히 나눌 수 있을 듯?
        cindex.append("분량")
        amount = soup2.find_all('p', class_='note')
        adic = {}
        for alist in amount:
            temp = alist.get_text()
            a = temp.split('\xa0\xa0|\xa0\xa0')            
            #print(temp)
            #if a[0] != '조회 결과가 없습니다.':
            if len(a) > 2:
                i_name = a[1].replace('성분명 : ','')
                i_amount = a[2].replace('분량 : ',' ')
                i_unit = a[3].replace('단위 : ', '')
                #print(i_name + i_amount + i_unit)
                adic[i_name] = i_amount+i_unit
        if adic == {}:
            datalist.append("")
        else:
            s = ""
            itemList = adic.items()
            for item in itemList:
                s += item[0] + " :" + item[1] + "\n"
            datalist.append(s)
            
        #효능효과 추가하기
        cindex.append("효능효과")
        way = soup2.find_all('div', id ='_ee_doc')
        s = ""
        for w in way:
            s += w.get_text()
        
        datalist.append(s)
        
        #용법용량 추가하기
        cindex.append("용법용량")
        way = soup2.find_all('div', id ='_ud_doc')
        s = ""
        for w in way:
            s += w.get_text()
        
        datalist.append(s)
        
        #주의사항 추가하기
        cindex.append("주의사항")
        way = soup2.find_all('div', id ='_nb_doc')
        s = ""
        for w in way:
            s += w.get_text()
        
        datalist.append(s)
        
            
        #URL 추가하기
        cindex.append("URL")
        datalist.append(detailURL)
        
        #추출 후 남은 데이터
        cindex.append("추출 후 남은 데이터")
        datalist.append("#none")
        
        #데이터 값 수정
        pos = cindex.index('품목기준코드')
        datalist[pos] = "kdrug-" + datalist[pos]
        
        #데이터 프레임 만들기
        temp = pd.DataFrame(data= [datalist],index=rindex, columns = cindex)
        
        #데이터 프레임 합치기
        df = pd.concat([df, temp])
        
        

df.to_excel("크롤링한 모든 데이터(의약품안전나라).xlsx", index = False )
print("전체 data 가져오기 성공")

  3%|██▋                                                                          | 250/7222 [20:18<8:05:59,  4.18s/it]

통신 에러
다시 request


  3%|██▋                                                                         | 252/7222 [20:49<18:27:25,  9.53s/it]

통신 에러
다시 request


  8%|█████▊                                                                    | 562/7222 [1:01:59<32:02:37, 17.32s/it]

통신 에러
다시 request
통신 에러
다시 request


  8%|█████▍                                                                | 563/7222 [2:22:48<2713:27:57, 1466.96s/it]

통신 에러
다시 request


  8%|█████▍                                                                | 564/7222 [2:23:08<1910:11:48, 1032.85s/it]

<h2>필요한 열만 추출하기

In [ ]:
finalDf = df[['품목기준코드','제품명','업체명','성상','유효성분','분량','효능효과','용법용량','주의사항','URL','추출 후 남은 데이터']]
finalDf.rename(columns={'품목기준코드':'제품코드','업체명':'제조사'},inplace=True)
finalDf.to_excel("result(의약품안전나라).xlsx", index = False)
print("최종 excel파일 생성 완료")

<h1>==================================================

<h2>너무 오래걸려서 함수로 만들어 나눠서 엑셀로 만듬

In [62]:
import requests
import json
import pandas as pd
import time
from bs4 import BeautifulSoup
from tqdm import tqdm

<h2>품목기준코드 URL 가져오기

In [63]:
def get_codeURL(start_pageNum, end_pageNum):
    code_List=[]
    for i in tqdm(range(int(start_pageNum),(int(end_pageNum)+1)) ) :
        itemSeqList = []
        requestURL = 'https://nedrug.mfds.go.kr/searchDrug?sort=&sortOrder=&searchYn=&ExcelRowdata=&page='+str(i)+'&searchDivision=detail&itemName=&entpName=&ingrName1=&ingrName2=&ingrName3=&itemSeq=&stdrCodeName=&atcCodeName=&indutyClassCode=&sClassNo=&narcoticKindCode=&cancelCode=&etcOtcCode=&makeMaterialGb=&searchConEe=AND&eeDocData=&searchConUd=AND&udDocData=&searchConNb=AND&nbDocData=&startPermitDate=&endPermitDate='
        while True:
            try:
                res = requests.get(requestURL)
                break
            except:
                print("통신 에러")
                print("다시 request")
                continue
        html_doc = res.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        #각 제품의 request보낼 url모으기
        links = soup.find_all("a")
        for a in links:
            href = a.attrs['href']
            if "itemSeq" in href:
                itemSeqList.append(href)
            itemNum = 15
            if i == 7222: itemNum=14 #마지막 페이지는 품목 14개임.
            if len(itemSeqList) == itemNum:
                break
        code_List += itemSeqList
    return code_List

<h2>페이지 당 품목 정보 엑셀로 만들기

In [64]:
def getData2Excel(itemSeqList, excelName):
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    for j in tqdm(range(0,len(itemSeqList))):
        cindex = []
        rindex = [str(j+1)]

        detailURL='https://nedrug.mfds.go.kr'+str(itemSeqList[j])
        while True:
            try:
                res2 = requests.get(detailURL)
                break
            except:
                print("통신 에러")
                print("다시 request")
                continue
        html_doc2 = res2.text
        soup2 =BeautifulSoup(html_doc2, 'html.parser')

        #col index 가져오기
        FILTER = ['년도','생산실적','순번','변경일자','변경항목'] #여기에 해당되는건 NO
        for title in soup2.find_all('th'):
            if(title.get_text() in (FILTER+cindex) ):
                break;
            cindex.append(title.get_text())
        datalist = []
        flag = 0
        for DATA in soup2.find_all('td'):
            #print(DATA.get_text())
            flag += 1
            datalist.append(DATA.get_text())
            if(flag == len(cindex)):
                break

        #유효성분 가져오기
        cindex.append("유효성분")
        ingredient = soup2.find_all('h3', class_='cont_title3 mt27 pb10')
        if ingredient:
            for ing in ingredient:
                datalist.append(ing.get_text().replace('유효성분 : ',''))
        else :
            datalist.append("")

        #유효성분 분량 가져오기
        #나중에 여기서 a[0]으로 세세히 나눌 수 있을 듯?
        cindex.append("분량")
        amount = soup2.find_all('p', class_='note')
        adic = {}
        for alist in amount:
            temp = alist.get_text()
            a = temp.split('\xa0\xa0|\xa0\xa0')            
            #print(temp)
            #if a[0] != '조회 결과가 없습니다.':
            if len(a) > 2:
                i_name = a[1].replace('성분명 : ','')
                i_amount = a[2].replace('분량 : ',' ')
                i_unit = a[3].replace('단위 : ', '')
                #print(i_name + i_amount + i_unit)
                adic[i_name] = i_amount+i_unit
        if adic == {}:
            datalist.append("")
        else:
            s = ""
            itemList = adic.items()
            for item in itemList:
                s += item[0] + " :" + item[1] + "\n"
            datalist.append(s)

        #효능효과 추가하기
        cindex.append("효능효과")
        way = soup2.find_all('div', id ='_ee_doc')
        s = ""
        for w in way:
            s += w.get_text()

        datalist.append(s)

        #용법용량 추가하기
        cindex.append("용법용량")
        way = soup2.find_all('div', id ='_ud_doc')
        s = ""
        for w in way:
            s += w.get_text()

        datalist.append(s)

        #주의사항 추가하기
        cindex.append("주의사항")
        way = soup2.find_all('div', id ='_nb_doc')
        s = ""
        for w in way:
            s += w.get_text()

        datalist.append(s)


        #URL 추가하기
        cindex.append("URL")
        datalist.append(detailURL)

        #추출 후 남은 데이터
        cindex.append("추출 후 남은 데이터")
        datalist.append("#none")

        #데이터 값 수정
        pos = cindex.index('품목기준코드')
        datalist[pos] = "kdrug-" + datalist[pos]

        #데이터 프레임 만들기
        temp = pd.DataFrame(data= [datalist],index=rindex, columns = cindex)

        #데이터 프레임 합치기
        df = pd.concat([df, temp])

    df.to_excel("모든 데이터["+str(excelName)+"](의약품안전나라).xlsx", index = False )
    print("전체 data 가져오기 성공")

    finalDf = df[['품목기준코드','제품명','업체명','성상','유효성분','분량','효능효과','용법용량','주의사항','URL','추출 후 남은 데이터']]
    finalDf.rename(columns={'품목기준코드':'제품코드','업체명':'제조사'},inplace=True)
    finalDf.to_excel("result["+str(excelName)+"](의약품안전나라).xlsx", index = False)
    print("최종 excel파일 생성 완료")
    return

In [59]:
temp = get_codeURL(1,10)
getData2Excel(temp, "1-10")

100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:59<00:00,  2.52it/s]


전체 data 가져오기 성공
최종 excel파일 생성 완료


<h1> 500페이지씩 엑셀로 출력

In [ ]:
interval = 500
for i in range(0,14):
    temp = get_codeURL(1 + (i*interval), 500 + (i*interval))
    getData2Excel(temp, str( 1 + (i*interval) ) + "-" + str( 500 + (i*interval) ) )

temp = get_codeURL(7001,7222)
getData2Excel(temp,"7001-7222")

 22%|█████████████████▏                                                            | 1658/7500 [10:42<28:42,  3.39it/s]

통신 에러
다시 request


 98%|████████████████████████████████████████████████████████████████████████████▋ | 7370/7500 [41:51<00:51,  2.51it/s]

통신 에러
다시 request


100%|██████████████████████████████████████████████████████████████████████████████| 7500/7500 [43:15<00:00,  2.89it/s]


...

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]

최종 excel파일 생성 완료


 16%|███████████▊                                                                | 1168/7500 [22:20<1:57:13,  1.11s/it]

통신 에러
다시 request
통신 에러
다시 request
통신 에러
다시 request
통신 에러
다시 request


 18%|█████████████▌                                                              | 1334/7500 [26:38<2:20:07,  1.36s/it]